# T5 + doc2query - Query Expansion

Author: Monique Monteiro (moniquelouise@gmail.com)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
main_dir = "/content/gdrive/MyDrive/Unicamp-aula-6-3"

## Dataset Download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz

--2023-04-05 22:04:11--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
Resolving huggingface.co (huggingface.co)... 18.67.0.90, 18.67.0.55, 18.67.0.67, ...
Connecting to huggingface.co (huggingface.co)|18.67.0.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/e9e97686e3138eaff989f67c04cd32e8f8f4c0d4857187e3f180275b23e24e85?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27corpus.jsonl.gz%3B+filename%3D%22corpus.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1680991452&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvZTllOTc2ODZlMzEzOGVhZmY5ODlmNjdjMDRjZDMyZThmOGY0YzBkNDg1NzE4N2UzZjE4MDI3NWIyM2UyNGU4NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9u

In [ ]:
!mv corpus.jsonl.gz {main_dir}/trec-covid

In [ ]:
!gunzip {main_dir}/trec-covid/corpus.jsonl.gz

In [ ]:
!head {main_dir}/trec-covid/corpus.jsonl

{"_id": "ug7v899j", "title": "Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia", "text": "OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60

## Libraries Installation

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.3 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.6 MB/s eta 0:00:00


In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.3 MB/s eta 0:00:00


## Model Prediction

Initial test. Parameters documentation at https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/text_generation#transformers.GenerationConfig.




In [ ]:
from torch.nn.modules import padding
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained(f"{main_dir}/doc2query-adafactor-bs-32-split-1000-bf16-early-stoping") 
#model_name = "doc2query-adafactor-bs-32-split-1000-no-fp16"
#model = T5ForConditionalGeneration.from_pretrained(f"{main_dir}/{model_name}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

input_ids = tokenizer(["Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital",
                       "Surfactant protein-D (SP-D) participates in the innate response to inhaled microorganisms and organic antigens, and contributes to immune and inflammatory regulation within the lung",
                       "Respiratory syncytial virus (RSV) and pneumonia virus of mice (PVM) are viruses of the family Paramyxoviridae, subfamily pneumovirus, which cause clinically important respiratory infections in humans and rodents, respectively"], 
                      return_tensors="pt",
                      padding="max_length"
                       ).input_ids.to(device)

sequence_ids = model.generate(input_ids, 
                              do_sample=True,
                              num_beams=1,
                              max_length=50, num_return_sequences=3
                              )
sequences = tokenizer.batch_decode(sequence_ids, skip_special_tokens=True)
sequences

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


['mycoplasma pneumoniae clinical features',
 'what disease can mycoplasma pneumoniae cause',
 'clinical features of mycoplasma pneumoniae infection at king alabama university',
 'what is polyp and additive protein-D',
 'what subunits are in sp-d',
 'what bacterium is called to act as a surfactant protein',
 'pneumoviruses disease',
 'parasites that cause respiratory infections',
 'what is respiratory syncytial virus (RSV)']

In [ ]:
def predict(texts, do_sample=False, num_beams=1, num_return_sequences=1):
  input_ids = tokenizer(texts, return_tensors="pt", 
                        padding="max_length", truncation=True).input_ids.to(device)
  sequence_ids = model.generate(input_ids, do_sample=do_sample,
                                num_beams=num_beams, max_new_tokens=128, 
                                num_return_sequences=num_return_sequences)
  sequences = tokenizer.batch_decode(sequence_ids, skip_special_tokens=True)
  return sequences

In [ ]:
import json

corpus = []

with open(f"{main_dir}/trec-covid/corpus.jsonl", 'r', encoding='utf-8') as f:
    # Read the file line by line
    lines = f.readlines()
    
    # Parse each line as a JSON object and store it in a list
    for line in lines:
        corpus.append(json.loads(line))


### Checking unprocessed documents

In [ ]:
import jsonlines
from tqdm.auto import tqdm
import glob
import pickle
import os

In [ ]:
#collections_dir = f"{main_dir}/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/"
#collections_dir = f"{main_dir}/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping/"
#collections_dir = f"{main_dir}/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries/"
#collections_dir = f"{main_dir}/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries/"
collections_dir = f"{main_dir}/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_5_queries_expand_titles/"

In [ ]:
processed_docs = set()
extension = "*.json"
files = glob.glob(os.path.join(collections_dir, extension))

for file in tqdm(files):
  with jsonlines.open(file) as reader:
    # Loop through each line in the file
    for item in reader:
        # Do something with the item, such as print it
        processed_docs.add(item["_id"])

0it [00:00, ?it/s]

In [ ]:
unprocessed_corpus = [doc for doc in corpus if doc['_id'] not in processed_docs]
len(unprocessed_corpus)

171332

### Query Expansion

In [ ]:
NUM_SAMPLE_QUERIES = 3

In [ ]:
import time
from collections import defaultdict
import pickle


batch_size = 32

doc_to_queries = defaultdict(list)

def add_queries(ids, queries, num_return_sequences=1):
  for i, q in enumerate(queries):
    doc_to_queries[ids[i]].append(q)
    
last_saved_batch = -1
extended_docs = []
gen_greedy = True
gen_beam = True

for i in tqdm(range(0, len(unprocessed_corpus), batch_size)):
  #texts = [doc["text"] for doc in unprocessed_corpus[i:i+batch_size]]
  #Last test: expand titles too
  texts = [doc["title"] + ' ' + doc["text"] for doc in unprocessed_corpus[i:i+batch_size]]
  ids = [doc["_id"] for doc in unprocessed_corpus[i:i+batch_size]]

  #Generates a batch of 1 greedy query per document
  if gen_greedy == True:
    greedy_queries = predict(texts)
    add_queries(ids, greedy_queries)
  
  if gen_beam == True:
    #Generates a batch of 1 query per document by using beam search
    beam_search_queries = predict(texts, num_beams=2)
    add_queries(ids, beam_search_queries)
  
  #Generates 3 queries per document by using random sampling
  sampled_queries = predict(texts, do_sample=True, 
                            num_return_sequences=NUM_SAMPLE_QUERIES)
  ids_list = []
  for id in ids:
    ids_list += [id]*NUM_SAMPLE_QUERIES

  add_queries(ids_list, sampled_queries, num_return_sequences=NUM_SAMPLE_QUERIES)    


  0%|          | 0/5355 [00:00<?, ?it/s]

In [ ]:
import pickle

with open(f'{main_dir}/unjsoned_5_queries_expand_titles.pickle', 'wb') as f:
  pickle.dump(doc_to_queries, f)

In [ ]:
import pickle

with open(f'{main_dir}/unjsoned_5_queries_expand_titles.pickle', 'rb') as f:
  doc_to_queries = pickle.load(f)

In [ ]:
len(doc_to_queries)

171332

## Indexing

### JSON serialization for indexing

In [ ]:
doc_id_to_text_and_title = {doc["_id"]: (doc["text"], doc["title"]) for doc in corpus}

Creates the object to be JSON-serialized.

In [ ]:
extended_docs = []

for doc_id, queries in doc_to_queries.items():
  text, title = doc_id_to_text_and_title[doc_id]
  text += '\n' + '\n'.join(queries)
  extended_docs.append({"id":doc_id, "contents": title + '\n' + text})

In [ ]:
len(extended_docs)

171332

In [ ]:
collections_dir

'/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_5_queries_expand_titles/'

## Index building in Pyserini

In [ ]:
json_batch_size = len(corpus)//10

In [ ]:
collections_dir

'/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_5_queries_expand_titles/'

In [27]:
collections_dir = '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_5_queries_expand_title'

In [28]:
j=0

for i in tqdm(range(0, len(extended_docs), json_batch_size)):
  filename = f"{collections_dir}/json_{j}.json"
  print(filename)
  with jsonlines.open(filename, mode='w') as writer:
    #for item in all_docs[i:i+json_batch_size]:
    for item in extended_docs[i:i+json_batch_size]:
      writer.write(item)
  j+=1

  0%|          | 0/11 [00:00<?, ?it/s]

/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_5_queries_expand_title/json_0.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_5_queries_expand_title/json_1.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_5_queries_expand_title/json_2.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_5_queries_expand_title/json_3.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_5_queries_expand_title/json_4.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_5_queries_expand_title/json_5.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2qu

In [29]:
#index_name = expanded_trec_covid
#index_name = "expanded_trec_covid_20_queries"
#index_name = "expanded_trec_covid_3_queries"
index_name = "expanded_trec_covid_5_queries_expand_title"

In [30]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input {collections_dir} \
  --index indexes/{index_name} \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-04-13 11:15:34,357 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-04-13 11:15:34,359 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-04-13 11:15:34,359 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-04-13 11:15:34,359 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_5_queries_expand_title
2023-04-13 11:15:34,360 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-04-13 11:15:34,360 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-04-13 11:15:34,360 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-04-13 11:15:34,360 INFO  [main] ind

In [31]:
!mv indexes/{index_name} {main_dir}